# Importante: Para utilizar esse arquivos é necessário ter acesso à pasta da sala e criar um atalho para a pasta "Grupo 1 - RappIA". Veja as instruções relevantes no Tópico 1 do notebook a seguir: https://colab.research.google.com/drive/1_l8X6shyH90NFIJso10Mw7Lq_NReoq6m#scrollTo=GrlId2ZHIoDF

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pickle 
from pickle import dump

# Carregar Scalers 
robust_scaler = pickle.load(open('/content/drive/MyDrive/Grupo 1 - RappIA /robust.scaler', 'rb'))
standard_scaler = pickle.load(open('/content/drive/MyDrive/Grupo 1 - RappIA /standard.scaler', 'rb'))
# Carregar Model
model = pickle.load(open('/content/drive/MyDrive/Grupo 1 - RappIA /rf.model', 'rb'))

In [ ]:
import pandas as pd

df = pd.read_csv('/content/drive/MyDrive/Grupo 1 - RappIA /features.csv', index_col='ID')
df.drop('Unnamed: 0', axis=1, inplace=True, errors='ignore')
df.drop('IS_CHURN', axis=1, inplace=True, errors='ignore')
df.drop('LAST_ORDER_DATE', axis=1, inplace=True, errors='ignore')
df.drop('ORDERS_CANCELED_LAST_30D_COUNT', axis=1, inplace=True, errors='ignore')
df.drop('ORDERS_LAST_30D_COUNT', axis=1, inplace=True, errors='ignore')
df.drop('SUPPORT_TICKET_COUNT', axis=1, inplace=True, errors='ignore')

In [ ]:
df.head(3)

,LEVEL_NAME,AUTO_ACCEPT,ACCEPTANCE_RATE,ORDERS_DONE,ORDERS_CANCEL,FIRST_ORDER_DATE,EARNINGS,TIPS,PUNISHMENT_MINUTES,PRODUCT_RETURNS,DEFECTS_COUNT,AVG_DISTANCE_TO_USER,TRANSPORT_bicycle,TRANSPORT_car,TRANSPORT_motorbike,TRANSPORT_neither,TRANSPORT_cargo_van,TRANSPORT_motorbike_trailer
ID,,,,,,,,,,,,,,,,,,
1552668,1,True,1.000,0,1,738354,38.75,0.0,15,0.0,2,3.765437,0,0,1,0,0,0
1552649,3,True,0.778,4,1,738353,626.51,203.0,45,0.0,8,4.014581,0,0,1,0,0,0
1552604,2,True,1.000,3,0,738353,143.56,59.0,60,0.0,9,1.517087,1,0,0,0,0,0


In [ ]:
df = pd.DataFrame(robust_scaler.transform(df), columns=df.columns, index=df.index)
df = pd.DataFrame(standard_scaler.transform(df), columns=df.columns, index=df.index)

preds = model.predict(df)

In [ ]:
print(preds.shape)
preds

(159528,)


array([ True, False, False, ..., False, False, False])

In [ ]:
preds_proba = model.predict_proba(df)

In [ ]:
print(preds_proba.shape)
preds_proba

(159528, 2)


array([[0.44335496, 0.55664504],
       [0.85409408, 0.14590592],
       [0.95559211, 0.04440789],
       ...,
       [0.85714286, 0.14285714],
       [0.82142857, 0.17857143],
       [0.75      , 0.25      ]])

In [ ]:
df['IS_CHURN'] = preds

In [ ]:
df['CHURN_PROBABILITY'] = preds_proba[:,1]

In [ ]:
df['CHURN_RANKING'] = df['CHURN_PROBABILITY'].apply(lambda x: 1 if x < 0.2 else 2 if x < 0.4 else 3 if x < 0.6 else 4 if x < 0.8 else 5)


In [ ]:
df

,LEVEL_NAME,AUTO_ACCEPT,ACCEPTANCE_RATE,ORDERS_DONE,ORDERS_CANCEL,FIRST_ORDER_DATE,EARNINGS,TIPS,PUNISHMENT_MINUTES,PRODUCT_RETURNS,...,AVG_DISTANCE_TO_USER,TRANSPORT_bicycle,TRANSPORT_car,TRANSPORT_motorbike,TRANSPORT_neither,TRANSPORT_cargo_van,TRANSPORT_motorbike_trailer,IS_CHURN,CHURN_PROBABILITY,CHURN_RANKING
ID,,,,,,,,,,,,,,,,,,,,,
1552668,0.565679,0.905204,1.039918,-0.127563,0.712107,684242.967652,29.967177,-0.196212,1.573333e+01,0.010729,...,3.373718,0.025645,-0.008512,0.981981,-0.000379,-0.003205,0.004232,True,0.556645,3
1552649,2.218602,0.905204,0.803152,3.537105,0.712107,684242.040938,487.315749,164.993248,4.704558e+01,0.010729,...,3.596439,0.025645,-0.008512,0.981981,-0.000379,-0.003205,0.004232,False,0.145906,1
1552604,1.392141,0.905204,1.039918,2.620938,-0.148497,684242.040938,111.522070,47.814518,6.270170e+01,0.010729,...,1.363818,1.034724,-0.008512,-0.017465,-0.000379,-0.003205,0.004232,False,0.044408,1
1552588,3.045063,0.905204,-0.026592,0.788604,-0.148497,684242.967652,317.047248,121.051224,1.566385e+02,0.010729,...,2.025013,0.025645,0.984496,-0.017465,-0.000379,-0.003205,0.004232,False,0.078178,1
1552553,0.565679,0.905204,0.921215,2.620938,-0.148497,684242.040938,103.359578,8.754941,2.254784e+07,0.010729,...,2.453931,0.025645,0.984496,-0.017465,-0.000379,-0.003205,0.004232,True,0.510702,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33189,1.392141,0.905204,0.146609,362.674605,14.481770,683919.544395,2454.647600,907.125205,2.254855e+07,-2.256669,...,1.862042,0.025645,0.984496,-0.017465,-0.000379,-0.003205,0.004232,False,0.142857,1
33188,-0.260782,-0.130537,0.927401,10.866442,0.712107,683997.388388,52.548257,17.706094,4.071365e+02,0.010729,...,3.734406,0.025645,-0.008512,0.981981,-0.000379,-0.003205,0.004232,True,0.588827,3
33161,1.392141,-0.130537,0.454617,2404.811050,47.184719,683917.690967,7419.022941,1948.713916,3.632298e+03,-76.562834,...,3.286615,0.025645,-0.008512,0.981981,-0.000379,-0.003205,0.004232,False,0.142857,1


In [ ]:
df[df['CHURN_RANKING'] == 1]

,LEVEL_NAME,AUTO_ACCEPT,ACCEPTANCE_RATE,ORDERS_DONE,ORDERS_CANCEL,FIRST_ORDER_DATE,EARNINGS,TIPS,PUNISHMENT_MINUTES,PRODUCT_RETURNS,...,AVG_DISTANCE_TO_USER,TRANSPORT_bicycle,TRANSPORT_car,TRANSPORT_motorbike,TRANSPORT_neither,TRANSPORT_cargo_van,TRANSPORT_motorbike_trailer,IS_CHURN,CHURN_PROBABILITY,CHURN_RANKING
ID,,,,,,,,,,,,,,,,,,,,,
1552649,2.218602,0.905204,0.803152,3.537105,0.712107,684242.040938,487.315749,164.993248,4.704558e+01,0.010729,...,3.596439,0.025645,-0.008512,0.981981,-0.000379,-0.003205,0.004232,False,0.145906,1
1552604,1.392141,0.905204,1.039918,2.620938,-0.148497,684242.040938,111.522070,47.814518,6.270170e+01,0.010729,...,1.363818,1.034724,-0.008512,-0.017465,-0.000379,-0.003205,0.004232,False,0.044408,1
1552588,3.045063,0.905204,-0.026592,0.788604,-0.148497,684242.967652,317.047248,121.051224,1.566385e+02,0.010729,...,2.025013,0.025645,0.984496,-0.017465,-0.000379,-0.003205,0.004232,False,0.078178,1
1552472,3.045063,0.905204,0.684236,14.531110,-0.148497,684242.040938,611.254909,142.208495,1.573333e+01,0.010729,...,3.408577,0.025645,-0.008512,0.981981,-0.000379,-0.003205,0.004232,False,0.128049,1
1552443,2.218602,0.905204,0.862344,1.704771,-0.148497,684241.114224,188.400454,71.413012,4.916101e+03,0.010729,...,1.277040,1.034724,-0.008512,-0.017465,-0.000379,-0.003205,0.004232,False,0.078178,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33203,3.045063,0.905204,0.497811,309.536914,10.178750,684091.913237,1775.456646,780.181580,2.959085e+03,0.010729,...,1.644287,1.034724,-0.008512,-0.017465,-0.000379,-0.003205,0.004232,False,0.178571,1
33197,1.392141,0.905204,0.206227,4.453272,3.293919,683978.854104,53.287473,12.823647,5.338816e+03,0.010729,...,3.041283,0.025645,0.984496,-0.017465,-0.000379,-0.003205,0.004232,False,0.145906,1
33189,1.392141,0.905204,0.146609,362.674605,14.481770,683919.544395,2454.647600,907.125205,2.254855e+07,-2.256669,...,1.862042,0.025645,0.984496,-0.017465,-0.000379,-0.003205,0.004232,False,0.142857,1


In [ ]:
df.to_excel('resultado_final.xlsx')
